# TO RUN EACH CELL JUST HOLD SHIFT + ENTER

### Import required packages

In [ ]:
import os
from IPython.display import Image
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re
import urllib.request
import time
import subprocess
from openai import OpenAI
import random
import requests
import glob

### Set necessary keys

In [ ]:
os.environ["OPENAI_API_KEY"] = ''

os.environ["SPOTIPY_CLIENT_ID"] = ""
os.environ["SPOTIPY_CLIENT_SECRET"] = ""
os.environ["SPOTIPY_REDIRECT_URI"] = ""

## Insert the name of your playlist as well as the share playlist link from Spotify

In [ ]:
playlist_name = input("Please enter the name of a playlist: ")
url = input("Please enter the share link to the playlist: ")

playlist_id_search = re.search(r'/playlist/([^/?]+)', url)
if playlist_id_search:
    playlist_id = playlist_id_search.group(1) 
    print(playlist_id)

## This gets all of the track names and artist names from your provided playlist

In [ ]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

def get_all_playlist_tracks(playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id) 
    tracks.extend(results['items'])

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    return tracks

all_tracks = get_all_playlist_tracks(playlist_id)

track_names = {}

for track in all_tracks:
    song_name = track['track']['name']
    artist_names = [artist['name'] for artist in track['track']['artists']]
    track_names[song_name] = artist_names

In [ ]:
tracks = list(track_names.keys())

all_songs = tracks.copy()

def choose_and_remove(track_names):
    if track_names:
        chosen = random.choice(track_names)
        track_names.remove(chosen)
        return chosen
    else:
        return None 

### Create a new folder for each playlist

In [ ]:
folder_path = os.path.join(os.getcwd(), 'images', playlist_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder was created.")
else:
    print("Folder already exists.")


### Figures out which song titles have already been generated

In [ ]:
folder_path = os.path.join(os.getcwd(), 'images', playlist_name)

jpg_files = glob.glob(os.path.join(folder_path, '*.jpg'))

created_images = []

for file in jpg_files:
    image_name = file.split(f"/images/{playlist_name}/")[1].split("_")[0]
    created_images.append(image_name)

print(created_images)

# This is the call to ChatGPT
## There are 4 things you can change  in this script:
### 1) You can change the "prompt=" section which is the prompt that will be given to DALL-E (Just change this one most!)
### 2) You can change the "size=" section. You won't actully want to do this though as it's already set to the highest resolution they allow
### 3) You can change the "quality=" section. Again you won't really want to do this, but if you want a smaller lower resolution image for some reason it can be set to "standard"
### 4) You can change the "n=" section. I haven't tested this with more than 1 image, so this might break the whole thing.

In [ ]:
### Figures out which song titles have already been generated
folder_path = os.path.join(os.getcwd(), 'images', playlist_name)
jpg_files = glob.glob(os.path.join(folder_path, '*.jpg'))

created_images = []
for file in jpg_files:
    image_name = file.split(f"/images/{playlist_name}/")[1].split("_")[0]
    created_images.append(image_name)

tracks = all_songs.copy()

client = OpenAI()

for _ in all_songs:
    while True:
        chosen_song = choose_and_remove(tracks) 
        if chosen_song not in created_images:
            artist = track_names[chosen_song][0]
            artist = artist.replace("/", "_")
            break
    if not chosen_song: 
        print("All song images have been generated from this playlist!")
    else:
        while True: 
            try:
                response = client.images.generate(
                  model="dall-e-3",
                  # prompt=f'create an image that reflects the meaning of {chosen_song} by {artist} and include the song title in the image and make it 16x9 aspect ratio',
                  prompt=f'You are an artist tasked with creating an evocative image inspired by the lyrics and messages conveyed in a song. The song you are focusing on is {chosen_song} by {artist}. Your goal is to craft an image that captures the essence, emotions, and themes portrayed in the song.', 
                  size="1792x1024",
                  quality="hd",
                  n=1,
                )
                image_url = response.data[0].url
                print(f"Image URL: {image_url}")
                break  
            except Exceptions as e:
                print(f"Error generating image for '{chosen_song}': {e}")
        file_name = f'{chosen_song}_{artist}.jpg' 
        file_path = os.path.join(os.getcwd(), 'images', playlist_name, file_name)
        image_data = urllib.request.urlopen(image_url).read()
        with open(file_path, 'wb') as f:
            f.write(image_data)

# if not tracks and 'image_url' not in locals(): 
#     print("Failed to generate an image for any of the provided songs.")


In [ ]:
tracks

In [ ]:
all_songs